# Getting Started with Z3

[Z3](https://github.com/Z3Prover/z3) is a state-of-the-art **theorem prover** developed at Microsoft Research and used in many real-world applications such as: software/hardware verification and testing, constraint solving, analysis of hybrid systems, security, and geometrical problems.

The Z3 input format is an extension of the [SMT-LIB 2.0 standard](http://www.smtlib.org/), but **APIs** for common programming languages like `Python`, `Java`, `C`, and `.Net` are officially available in order to **ease implementation** and favor its use as a component in the context of other tools that require solving logical formulas. Unoffical portings for functional languages like [`Scala`](https://github.com/epfl-lara/ScalaZ3) or [`Haskell`](https://github.com/LeventErkok/sbv) are also available

For the sake of simplicity, in this tutorial we will explore how to use **Z3Py** (the Z3 API in Python), which is the programming language the reader is probably most confortable with. 

At the time of writing, the latest version of Z3 is the `4.8.X`, which is the one used here. Any compatibility issues you may find running this notebook can be related to a different version of the tool.

## Installation

The easiest way to install Z3Py, along with the Z3 binary, is to use Python's package manager `pip`.

```bash
pip install z3-solver
```

In order to check if Z3 has been installed properly, open a terminal and type:

```bash
z3 --version
```

If the output is something like _Z3 version 4.8.X - 64 bit_ you are good to go.

In [1]:
from z3 import *

## Boolean Logic

Z3 obviously supports boolean operators: `And`, `Or`, `Not`, `Implies`, `If`, `==` (bi-implication). 

The following example shows how to create a boolean formula.

In [2]:
p = Bool('P')
q = Bool('Q')
r = Bool('R')

formula = And(Implies(p, q), r == Not(q), Or(Not(p), r))
formula

And(Implies(P, Q), R == Not(Q), Or(Not(P), R))

You can use the `solve` method to find an assignment of the variables which satisfies the formula.

In [3]:
solve(formula)

[R = False, Q = True, P = False]


`And` operator is implicitly considered by the `solve` function because all the clauses must be valid at the same time, threfore we can omit it.

In [4]:
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

[R = False, Q = True, P = False]


## Arithmetic

Z3 supports real and integer variables. They can be mixed in a single problem. Like most programming languages, Z3Py will automatically add coercions converting integer expressions to real ones when needed.

In [5]:
a, b, c = Ints('a b c')
d, e, f, g = Reals('d e f g')

constraints = [
    a + b - 350 == 0,
    a - g == 0,
    c - 400 == 0,
    b - d * 0.45 == 0,
    c - f - e - d == 0,
    d <= 250,
    e <= 250
]

solve(constraints)

[e = 250, b = 0, f = 150, d = 0, c = 400, g = 350, a = 350]


Let's see a practical use case from high school: kinematic equations. These equations describe the mathematical relationship between displacement (`d`), time (`t`), acceleration (`a`), initial velocity (`v_i`) and final velocity (`v_f`):

$$d = v_i \cdot t + \frac{1}{2}a \cdot t^2$$

$$v_f = v_i + a \cdot t$$

Providing Z3 with a formulary and the available data from a problem instance, we can easily solve it.

In [6]:
d, a, t, v_i, v_f = Reals('d a t v__i v__f')

equations = [
   d == v_i * t + (a*t**2)/2,
   v_f == v_i + a*t,
]
print("Kinematic equations:")
print(equations)

# Given v_i, v_f and a, find d
problem1 = [
    v_i == 30,
    v_f == 0,
    a   == -8
]

# Given v_i, t and a, find d
problem2 = [
    v_i == 0,
    t   == 4.10,
    a   == 6
]

Kinematic equations:
[d == v__i*t + (a*t**2)/2, v__f == v__i + a*t]


In [7]:
problem = problem1

print("\nProblem:")
print(problem)

print("\nSolution:")
solve(equations + problem)


Problem:
[v__i == 30, v__f == 0, a == -8]

Solution:
[a = -8, v__f = 0, v__i = 30, t = 15/4, d = 225/4]


### Machine Arithmetic

Machine arithmetic is available in Z3Py as Bit-Vectors. They implement the precise semantics of unsigned and of signed two-complements arithmetic.

Some low level hacks are very popular among programmers. We can prove the correctness of these tricks using Z3.

Here for example we will exploit the fact that powers of 2 can be identified obtaining all 0s when putting in `And` two consecutive binary numbers.

> Note: Pay attention to the 0, it would be incorrectly classified as a power of 2 if not explicitly excluded.

We will discuss about `prove` method in the following section, just consider it as a way of determining the validity of a formula.

In [8]:
x = BitVec('x', 32)
powers = [ 2**i for i in range(32) ]
method1 = And(x != 0, x & (x - 1) == 0)
method2 = Or([ x == p for p in powers ])

prove(method1 == method2)

proved


Just slightly changing `method1`, for instance omitting the special case of the 0, the formula does not hold anymore.

In [9]:
method1 = x & (x - 1) == 0
prove(method1 == method2)

counterexample
[x = 0]


## Solvers

The command `solve`, used in the previous examples, is implemented using the Z3 solver API.

The command `Solver` creates a general purpose solver. 

In [10]:
x = Int('x')
y = Int('y')

s = Solver()
s

[]

Constraints can be added using the method `add`. We say the constraints have been asserted in the solver.

In [11]:
s.add(x > 10, y == x + 2)
s

[x > 10, y == x + 2]

The method `check` solves the asserted constraints. The result is `sat` (satisfiable) if a solution was found, `unsat` (unsatisfiable) if no solution exists, or `unknown` if the solver fails to return for some other reason...

In [12]:
print("Solving constraints in the solver s...")
print(s.check())
s.model()

Solving constraints in the solver s...
sat


[x = 11, y = 13]

... one particularly common is the situation of `timeout`.

In [13]:
time = 10 # in milliseconds
s.set(timeout=time)

if s.check() == sat:
    print(s.model())
elif s.reason_unknown() == "timeout":
    print("Solver timeout")

[x = 11, y = 13]


In some applications, we want to explore several similar **problems** that **share** several **constraints**. We can use the commands `push` and `pop` for doing that. 

Each **solver maintains** a **stack of assertions**. The command `push` creates a **new scope** by saving the current stack size. The command `pop` **removes** any assertion performed between it and the **matching push**. The `check` method always operates on the content of solver assertion stack.

In [14]:
print("Create a new scope...")
s.push()
s.add(y < 11, x > 1)
s

Create a new scope...


[x > 10, y == x + 2, y < 11, x > 1]

In [15]:
print("Solving updated set of constraints...")
print(s.check())

Solving updated set of constraints...
unsat


In [16]:
print("Restoring state...")
s.pop()
print(s.check())
s.model()

Restoring state...
sat


[y = 13, x = 11]

Finally, we can be interested in finding more than one feasible solution for the same formula.

Z3 does not provide an explicit method to do so, but you can use the following trick:

In [17]:
def solve_multiple(s, n=None):
    s.push()
    i = 0
    while s.check() == sat and (n is None or i < n):
        print(s.model())
        s.add(Or(x != s.model()[x], y != s.model()[y])) # prevent next model from using the same assignment as a previous model
        i += 1
    s.pop()

solve_multiple(s, n=10)

[y = 13, x = 11]
[y = 14, x = 12]
[y = 15, x = 13]
[y = 16, x = 14]
[y = 17, x = 15]
[y = 18, x = 16]
[y = 19, x = 17]
[y = 20, x = 18]
[y = 21, x = 19]
[y = 22, x = 20]


### Proofs

Until now, we have talked about **satisfiability**, determining an assignment of appropriate values under which our series of constraints evaluates to true.

Instead, we can be interested in finding a **proof** of a statement _F_, i.e., whether it is **valid for any combination** of values. In order to do that, we can exploit the fact that if _F_ is always true, `Not`(_F_) is always false, therefore trying to satisfy it gives `unsat` as result.

The following example use this trick to prove De Morgan's law.

In [18]:
p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))
demorgan

And(p, q) == Not(Or(Not(p), Not(q)))

In [19]:
print("Proving De Morgan...")
prove(demorgan)

Proving De Morgan...
proved


### Optimization

In the _Arithmetic_ section we have taken into consideration a system of constraints, and the solver provided a feasible assignment of all the variables involved. However, in common real case scenarios, also an objective cost dependent on the involved variables is provided, and your aim is to minimize/maximize it.

Let's use the same set of constraints and add an `objective`.

In [20]:
cost = Real('cost')
objective = cost == f * 50 + e * 40 + d * 20 + g * 50

s = Solver()
s.add(constraints + [objective])
s.check()
s.model()

[e = 250,
 b = 0,
 cost = 35000,
 f = 150,
 d = 0,
 c = 400,
 g = 350,
 a = 350]

The assignment is the same we previously obtained but here we have also determined the corresponding `cost`. What we are not aware of is the quality of our solution.

`Optimize` is a different type of solver to which you can specify the variable to minimize/maximize.

In [21]:
opt = Optimize()
opt.add(constraints + [objective])

h = opt.minimize(cost)
opt.check()
opt.lower(h)
opt.model()

[b = 112,
 e = 250,
 cost = 65800/3,
 f = -890/9,
 d = 2240/9,
 c = 400,
 g = 238,
 a = 238]

Comparing the `cost` values of the two solutions we can clearly see the benefits of using the latter approach when you are interested in the quality of your solution and not only on its feasibility.

## Additional Resources

* [Official repo](https://github.com/z3prover/z3)
* [SMT-LIB 2.0 tutorial](https://rise4fun.com/z3/tutorial)
* [More examples using Z3Py](https://ericpony.github.io/z3py-tutorial/)
* [Mastering Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html)